In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install clearml
!pip install -U sentence-transformers
!pip install datasets
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.5 MB/s eta 0:00:00
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling PyJWT-2.3.0:
      Successfully uninstalled PyJWT-2.3.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=83353049b25cf1457e714467547150c166ac7ac7130d58d5b49dbbca5dfd9eba
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:0

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, models, evaluation, losses, util
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np
import torch
import torch.nn.functional as F
from torch.optim import AdamW
from transformers import AutoTokenizer
from datasets import load_dataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
from clearml import Task

Task.set_credentials(
    api_host='https://api.clear.ml',
    web_host='https://app.clear.ml',
    files_host='https://files.clear.ml',
    key='KEY',
    secret='SECRET'
)

In [ ]:
# Инициализация задачи в ClearML
task = Task.init(project_name='HuggingFace Transformers', task_name='SentenceTransformer')
logger = task.get_logger()

ClearML Task: created new task id=3dae9bdeccf944c98b484dae943c34d5
2023-12-22 12:28:39,812 - clearml.Task - INFO - No repository found, storing script code instead
ClearML results page: https://app.clear.ml/projects/168ff058d56740e39649f6706a560d33/experiments/3dae9bdeccf944c98b484dae943c34d5/output/log


In [ ]:
dataset = load_dataset('csv', data_files='/content/drive/MyDrive/Colab Notebooks/combined_data.csv', delimiter='|')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['vacancy', 'cv', 'response'],
        num_rows: 336061
    })
})

In [ ]:
train_dataset, test_dataset = dataset['train'].train_test_split(test_size=0.01).values()

In [ ]:
train_examples = []
n_examples = train_dataset.num_rows
for i in range(n_examples):
    example = train_dataset[i]
    train_examples.append(InputExample(texts=[example['vacancy'], example['cv']], label=example['response']))

In [ ]:
test_examples = []
n_examples = test_dataset.num_rows
for i in range(n_examples):
    example = test_dataset[i]
    test_examples.append(InputExample(texts=[example['vacancy'], example['cv']], label=example['response']))

test_vacancy, test_cv, test_label = [], [], []
n_examples = test_dataset.num_rows
for i in range(n_examples):
    example = test_dataset[i]
    test_vacancy.append(example['vacancy'])
    test_cv.append(example['cv'])
    test_label.append(example['response'])

In [ ]:
# Загрузка модели SentenceTransformer
model_id = "cointegrated/rubert-tiny2"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# word_embedding_model = models.Transformer(model_id)
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
# model.to(device)
model = SentenceTransformer(model_id)

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

.gitignore:   0%|          | 0.00/6.00 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

...

tinybert-ru-labse-adapter-v2.pt:   0%|          | 0.00/963k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
def collate_fn(batch):
    texts = [example.texts for example in batch]  # Получение текстов
    labels = torch.tensor([example.label for example in batch])  # Получение меток
    return texts, labels

# Подготовка данных
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=4, collate_fn=collate_fn)
test_dataloader = DataLoader(test_examples, shuffle=False, batch_size=4, collate_fn=collate_fn)

In [ ]:
# train_loss = losses.CosineSimilarityLoss(model)
train_loss = losses.ContrastiveLoss(model)

In [ ]:
# evaluator = evaluation.EmbeddingSimilarityEvaluator(test_vacancy,
#                                                     test_cv,
#                                                     test_label,
#                                                     batch_size=4,
#                                                     main_similarity=evaluation.SimilarityFunction.COSINE,
#                                                     show_progress_bar=True,
#                                                     write_csv=True,
#                                                     name="cosine")
evaluator = evaluation.BinaryClassificationEvaluator(test_vacancy,
                                                     test_cv,
                                                     test_label,
                                                     batch_size=4,
                                                     show_progress_bar=True,
                                                     write_csv=True,
                                                     name="cosine")

In [ ]:
num_epochs = 1
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)

In [ ]:
for i, (texts, labels) in enumerate(test_dataloader):
    embeddings1 = torch.tensor(model.encode([x[0] for x in texts]))
    embeddings2 = torch.tensor(model.encode([x[1] for x in texts]))
    cosine_similarities = F.cosine_similarity(embeddings1, embeddings2)
    predictions = (cosine_similarities > 0.5).int().tolist()
    break

In [ ]:
def callback(score, epoch, steps):

    logger.report_scalar("Evaluation", "Accuracy", score, steps)
    torch.save(model.state_dict(), 'last.pt')
    task.upload_artifact(f'sentence_transformer_{steps}_{score}', artifact_object='last.pt')
    all_predictions, all_labels = [], []
    for i, (texts, labels) in enumerate(test_dataloader):
        embeddings1 = torch.tensor(model.encode([x[0] for x in texts]))
        embeddings2 = torch.tensor(model.encode([x[1] for x in texts]))
        cosine_similarities = F.cosine_similarity(embeddings1, embeddings2)
        predictions = (cosine_similarities > 0.5).int().tolist()
        all_predictions.extend(predictions)
        all_labels.extend(labels.tolist())

    overall_precision = precision_score(all_labels, all_predictions)
    overall_recall = recall_score(all_labels, all_predictions)
    overall_f1 = f1_score(all_labels, all_predictions)

    logger.report_scalar("Evaluation", "precision", overall_precision, steps)
    logger.report_scalar("Evaluation", "recall", overall_recall, steps)
    logger.report_scalar("Evaluation", "f1", overall_f1, steps)

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          evaluator=evaluator,
          output_path="sentence_transformer",
          save_best_model=True,
          callback=callback,
          evaluation_steps=5000)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/83175 [00:00<?, ?it/s]

Batches:   0%|          | 0/1501 [00:00<?, ?it/s]

2023-12-22 12:51:53,520 - clearml.frameworks - INFO - Found existing registered model id=3c1fc67891b045d88b088ce92147418b [/content/last.pt] reusing it.


Batches:   0%|          | 0/1501 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1501 [00:00<?, ?it/s]

Batches:   0%|          | 0/1501 [00:00<?, ?it/s]

Batches:   0%|          | 0/1501 [00:00<?, ?it/s]

Batches:   0%|          | 0/1501 [00:00<?, ?it/s]

Batches:   0%|          | 0/1501 [00:00<?, ?it/s]

In [ ]:
def metrics_with_threshold(threshold):
    all_predictions, all_labels = [], []
    for i, (texts, labels) in enumerate(test_dataloader):
        embeddings1 = torch.tensor(model.encode([x[0] for x in texts]))
        embeddings2 = torch.tensor(model.encode([x[1] for x in texts]))
        cosine_similarities = F.cosine_similarity(embeddings1, embeddings2)
        predictions = (cosine_similarities > threshold).int().tolist()
        all_predictions.extend(predictions)
        all_labels.extend(labels.tolist())

    print(f"Threshold: {threshold}")
    print(f"Precision: {precision_score(all_labels, all_predictions)}")
    print(f"Recall: {recall_score(all_labels, all_predictions)}")
    print(f"F1: {f1_score(all_labels, all_predictions)}")
    print('------------------------------------------------')

In [ ]:
thresholds = [0.5, 0.6, 0.7, 0.8, 0.9]
for threshold in thresholds:
    metrics_with_threshold(threshold)

Threshold: 0.5
Precision: 0.6351560086499846
Recall: 0.9985429820301117
F1: 0.7764350453172205
------------------------------------------------
Threshold: 0.6
Precision: 0.7483320978502594
Recall: 0.9805730937348227
F1: 0.848854319949548
------------------------------------------------
Threshold: 0.7
Precision: 0.8138655462184874
Recall: 0.9407479358912093
F1: 0.8727190808740708
------------------------------------------------
Threshold: 0.8
Precision: 0.8660968660968661
Recall: 0.8858669256920836
F1: 0.8758703481392558
------------------------------------------------
Threshold: 0.9
Precision: 0.9056716417910448
Recall: 0.736765420106848
F1: 0.8125334761649706
------------------------------------------------


In [ ]:
task.close()